# AI Tutor Debugging Session

This notebook is designed for step-by-step debugging of the AI Tutor's core feedback loop. Each cell represents a single API call, allowing for careful inspection of the requests and responses.

In [44]:
import requests
import json
import time

# --- Configuration ---
BASE_URL = "http://127.0.0.1:8000"
USER_ID = f"debug_user_{int(time.time())}"
QUESTION_NUMBER = 3 # A question we know the answer to
INCORRECT_ANSWER = "1" # The 1-based index for 'δ (delta)'
CORRECT_ANSWER = "2" # The 1-based index for 'ε (epsilon)'

session = requests.Session()
hint_data = {} # To store hint response

print(f"Using User ID: {USER_ID}")

Using User ID: debug_user_1767656021


## Step 1: Create the User

In [45]:
payload = {"user_id": USER_ID}
response = session.post(f"{BASE_URL}/users/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "user_id": "debug_user_1767656021",
  "created_at": "2026-01-05T23:33:44.474543",
  "preferences": {
    "hint_style_preference": "adaptive",
    "intervention_preference": "manual"
  },
  "feedback_scores": {},
  "skill_mastery": [],
  "interaction_history": []
}


## Step 2: Set User Preferences

In [48]:
payload = {
    "intervention_preference": "proactive",
    "hint_style_preference": "adaptive" #"Socratic Question"
}
response = session.put(f"{BASE_URL}/users/{USER_ID}/preferences/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "hint_style_preference": "adaptive",
  "intervention_preference": "proactive"
}


## Step 3: Submit an INCORRECT First Answer

In [49]:
payload = {
    "user_id": USER_ID,
    "question_number": QUESTION_NUMBER,
    "user_answer": INCORRECT_ANSWER
}
response = session.post(f"{BASE_URL}/answer/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "correct": false,
  "correct_answer": "2",
  "skill": "[LoRA-Quantization-Basics]",
  "intervention_needed": true,
  "current_mastery": 0.17575757575757575
}


# Step 3.5: send a skipped answer

In [50]:
payload = {
        "user_id": USER_ID,
        "question_number": QUESTION_NUMBER,
        "skipped": True
    }
response = session.post(f"{BASE_URL}/answer/", json=payload)
 
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "correct": false,
  "correct_answer": "2",
  "skill": "[LoRA-Quantization-Basics]",
  "intervention_needed": true,
  "current_mastery": 0.17575757575757575
}


## Step 4: Request a Hint

This is the most critical step. We need to inspect the full response body to see if `context` and `final_prompt` are being populated correctly.

In [83]:
payload = {
    "user_id": USER_ID,
    "question_number": QUESTION_NUMBER,
    "user_answer": INCORRECT_ANSWER
}
response = session.post(f"{BASE_URL}/hints/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

if response.ok:
    hint_data = response.json()

Status Code: 200
{
  "question_number": 3,
  "hint": "Let's consider a similar problem to understand how quantization error can negatively impact LoRA adapter training.\n\n**Problem:** How does quantization error affect the LoRA adapter's ability to learn?\n\nHere's a step-by-step breakdown:\n\n1.  **The Ideal Scenario (No Quantization):** In a perfect world, without quantization, the LoRA adapter learns to adjust the full-precision base model's weights to improve performance on a specific task. The adapter receives",
  "user_id": "debug_user_1767656021",
  "hint_style": "Worked Example",
  "pre_hint_mastery": 0.17142857171212977,
  "context": "behavior is perturbed by quantization noise; the adapter must \u201cwork around\u201d that.\u200b\n \n\u25cf\u200b QA-LoRA\u2019s insight: make the LoRA adaptation aware of quantization during training, \nnot just afterward. It blends quantization and adaptation more tightly. (Hugging Face)\u200b\n \nIn practice, QA-LoRA provides two advantages:

## Step 5: Submit a CORRECT Second Answer (with Hint Context)

In [ ]:
if not hint_data:
    print("Cannot proceed: hint_data was not populated in the previous step.")
else:
    payload = {
        "user_id": USER_ID,
        "question_number": QUESTION_NUMBER,
        "user_answer": CORRECT_ANSWER,
        "hint_shown": True,
        "hint_style_used": hint_data.get("hint_style"),
        "hint_text": hint_data.get("hint"),
        "pre_hint_mastery": hint_data.get("pre_hint_mastery"),
        "feedback_rating": 4 # Assume a good hint
    }
    response = session.post(f"{BASE_URL}/answer/", json=payload)

    print(f"Status Code: {response.status_code}")
    print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "correct": false,
  "correct_answer": "2",
  "skill": "[LoRA-Quantization-Basics]",
  "intervention_needed": true,
  "current_mastery": 0.17142857171212977
}
